# Zonal Stats

By Cascade Tuholske, June 2020

Notebook finds zonal stats of populations for give geographies. <br><br>
**NOTE** CRS should be epsg:4326 for everything!

In [ ]:
#### Dependencies
import numpy as np
import pandas as pd
import rasterio
import geopandas as gpd
from rasterstats import zonal_stats, gen_zonal_stats
from glob import glob

In [ ]:
#### File Paths & FNs
DATA_PATH = '/Users/cascade/Github/PopGridCompare/data/'

In [ ]:
#### Run on Nigeria GDAM 
polys_fn = DATA_PATH+'raw/GDAM/gadm36_ESP_shp/gadm36_ESP_4.shp'
polys = gpd.read_file(polys_fn)
col = 'GID_4' # gdam level
fn_out = DATA_PATH+'interim/ESP4_stats.shp' #updatea

In [ ]:
# subset, be sure to check the admin level
polys = polys[['geometry', col]]

In [ ]:
# Git tif files
rst_fns = glob('/Users/cascade/Github/PopGridCompare/data/interim/*.tif')
rst_fns

In [ ]:
def zone_loop(polys_in, rst_list, stats_type):
    """ Function loops through rasters, calcs zonal_stats and returns stats as a data frame.
    Args:
        polys_in = polygons
        rst_list = list of paths & fns of rasters
        stats_type = stats type for each poly gone (see zonal stats)
    """
    
    # copy polys to write out
    polys_out = polys_in.copy()
    
    for rst in rst_list:
        
        # Get data name
        data = rst.split(DATA_PATH+'interim/')[1].split('_matched.tif')[0]
        print('Started', data)
        
        # Run zonal stats
        zs_feats = zonal_stats(polys_in, rst, stats=stats_type, geojson_out=True)
        zgdf = gpd.GeoDataFrame.from_features(zs_feats, crs=polys_in.crs)
        
        # Rename columns and merge
        zgdf = zgdf.rename(columns={stats_type: data+'_'+stats_type})
        
        polys_out = polys_out.merge(zgdf[[col, data+'_'+stats_type]], on = col, how = 'inner')
    
    return polys_out


In [ ]:
# Run zonal stats loop
polys_sum = zone_loop(polys, rst_fns, 'sum')
polys_sum.head()

In [ ]:
# variances
polys_sum['var'] = polys_sum.iloc[:,2:6].var(axis = 1)

In [ ]:
# std (population level)
polys_sum['std'] = polys_sum.iloc[:,2:6].std(axis = 1, ddof=0)

In [ ]:
# range
polys_sum['range'] = polys_sum.iloc[:,2:6].max(axis = 1) - polys_sum.iloc[:,2:6].min(axis = 1)

In [ ]:
polys_sum.head()

In [ ]:
#### Save it out
gpd_out = gpd.GeoDataFrame(polys_sum)
gpd_out.to_file(fn_out)

# Old Code

In [ ]:
#### Functions
def zonal_func(polys_in, rst_in, stats_type, save, fn_out = None):
    """ Runs zonal stats on a set of polygons for a given raster, see rasterstats for stats type.
    Returns geodata frame
    Args:
        polys = polygons as a shape file read into memory
        rst_fn = path to raster file to run zonal stats on
        stats_type = stats type for each poly gone (see zonal stats)
        save = True will save out a fail
        fn_out = file name and path to save out shape files
    """
    
    # Run Zonal Stats & Set to gpd df
    zs_feats = zonal_stats(polys_in, rst_in, stats=stats_type, geojson_out=True)
    zgdf = gpd.GeoDataFrame.from_features(zs_feats, crs=polys_in.crs)
    
    if save == True:
        zgdf.to_file(fn_out) 
    
    return zgdf

In [ ]:
#### Run Zonal Stats
rst_fns = glob('/Users/cascade/Github/PopGridCompare/data/interim/*.tif')
fn_out = DATA_PATH+'interim/NGA_2_LS15.shp'
rst_fn = rst_fns[0]
zstats = zonal_func(polys, rst_fn, 'sum', save = True, fn_out =fn_out)